# Juliaで100本ノック(26-50)

## 準備

In [ ]:
ENV["COLUMNS"]=240  # 描画する表の列数を増やす
ENV["LINES"]=10  # 行の数は制限（問題の指示とは異なるので好みに合わせて修正）

using Pkg

Pkg.add("DataFrames")
Pkg.add("DataFramesMeta")
Pkg.add("LibPQ")
Pkg.add("StatsBase")

using DataFrames
using DataFramesMeta
using LibPQ
using StatsBase
using Statistics
using Dates

## SQLとの接続

In [ ]:
host = "db"
port = ENV["PG_PORT"]
database = ENV["PG_DATABASE"]
user = ENV["PG_USER"]
password = ENV["PG_PASSWORD"]
dsl = "postgresql://$user:$password@$host:$port/$database"
conn = LibPQ.Connection(dsl)

df_customer = DataFrame(execute(conn, "select * from customer"))
df_category = DataFrame(execute(conn, "select * from category"))
df_product = DataFrame(execute(conn, "select * from product"))
df_receipt = DataFrame(execute(conn, "select * from receipt"))
df_store = DataFrame(execute(conn, "select * from store"))
df_geocode = DataFrame(execute(conn, "select * from geocode"));

## 本編

### 026

In [ ]:
# より本格的にlinqに頼っていく
@linq df_receipt |>
    select(:customer_id, :sales_ymd) |>
    orderby(:customer_id, :sales_ymd) |>
    groupby(:customer_id) |>
    combine(:sales_ymd => last, :sales_ymd => first) |>
    where(:sales_ymd_last .!= :sales_ymd_first) |>
    first(10)

### 027

In [ ]:
@linq df_receipt |>
    select(:store_cd, :amount) |>
    groupby(:store_cd) |>
    combine(:amount => mean) |>
    sort(:amount_mean, rev=true) |>
    first(5)

### 028

In [ ]:
# Statisticsライブラリ内にmedianがあるので使うだけ
@linq df_receipt |>
    select(:store_cd, :amount) |>
    groupby(:store_cd) |>
    combine(:amount => median) |>
    sort(:amount_median, rev=true) |>
    first(5)

### 029

In [ ]:
# 同様にmodeを使うだけ
@linq df_receipt |>
    select(:store_cd, :product_cd) |>
    groupby(:store_cd) |>
    combine(:product_cd => mode) |>
    orderby(:store_cd) |>
    first(10)

### 030

In [ ]:
@linq df_receipt |>
    select(:store_cd, :amount) |>
    groupby(:store_cd) |>
    combine(:amount => var) |>
    sort(:amount_var, rev=true) |>
    first(5)

### 031

In [ ]:
# stdは標準ライブラリとStatsBaseの両方に入っていて、前者ではデフォルトが不偏分散、後者では標本分散になるのでまぎらわしい。
# ここでは匿名関数経由でcorrected=falseを明示的に渡して紛れがないようにする。
uncorrected_std(x) = std(x, corrected=false)
@linq df_receipt |>
    select(:store_cd, :amount) |>
    groupby(:store_cd) |>
    combine(:amount => uncorrected_std) |>
    sort(:amount_uncorrected_std, rev=true) |>
    first(5)

### 032

In [ ]:
# StatsBaseのnquantileを使う。Indexがなくなるので適当に作ってつなげてやる。
hcat(Array(0:4)./4, nquantile(df_receipt.amount, 4))

### 033

In [ ]:
# 27番をほぼそのまま流用するだけ
@linq df_receipt |>
    select(:store_cd, :amount) |>
    groupby(:store_cd) |>
    combine(:amount => mean) |>
    sort(:amount_mean, rev=true) |>
    where(:amount_mean .>= 330)

### 034

In [ ]:
# もし正規表現の中で否定するのは可読性の点で今一つなものの、whereの全体を否定する方法がないようなのでしかたなく
@linq df_receipt |>
    select(:customer_id, :amount) |>
    where(occursin.(r"^[^Z]", :customer_id)) |>
    groupby(:customer_id) |>
    combine(:amount => sum) |>
    combine(:amount_sum => mean)

### 036

In [ ]:
# あとから列を選ぶのは手間なのであらかじめdf_storeの列をしぼってからjoinする。
# ソートは問題に指示されていないため行わない（Pythonの回答はソートされているが）
first(innerjoin(df_receipt,
                @select(df_store, :store_cd, :store_name),
                on=:store_cd), 10)

### 037

In [ ]:
first(innerjoin(df_product,
                df_category[:, [:category_small_cd, :category_small_name]],
                on=:category_small_cd), 10)

### 038

In [ ]:
# gender_cdがmissingのものがあるので最初に始末。そのあとcoalesce.でmissingを0に置き換える。
@linq outerjoin(df_customer[:, [:customer_id, :gender_cd]],
                df_receipt[:, [:customer_id, :amount]],
                on=:customer_id) |>
    dropmissing(:gender_cd) |>
    where(:gender_cd .== "1") |>
    where(occursin.(r"^[^Z]", :customer_id)) |>
    groupby(:customer_id) |>
    combine(:amount => sum) |>
    transform(amount_sum = coalesce.(:amount_sum, 0)) |>
    first(10)

### 039

In [ ]:
df_sum = @linq df_receipt |>
    select(:customer_id, :amount) |>
    where(occursin.(r"^[^Z]", :customer_id)) |>
    groupby(:customer_id) |>
    combine(:amount => sum) |>
    sort(:amount_sum, rev=true) |>
    first(20)

df_cnt = @linq df_receipt |>
    select(:customer_id, :sales_ymd) |>
    where(occursin.(r"^[^Z]", :customer_id)) |>
    unique() |>  # すでに二つの列にしぼってあるので全体でuniqifyしてよい
    groupby(:customer_id) |>
    combine(:sales_ymd => length) |>
    sort(:sales_ymd_length, rev=true) |>
    first(20)

outerjoin(df_sum, df_cnt, on=:customer_id)

### 040

In [ ]:
# Pythonでの回答はずいぶん手がこんでいるが、単純にこれでよい？
nrow(df_product) * nrow(df_store)

### 041

In [ ]:
# Pythonでの回答もdiffを使ったほうが簡単だと思う。
# diffだと一つ短い配列になるのでmissingをvcatして埋めてやる。
@linq df_receipt |>
    select(:sales_ymd, :amount) |>
    groupby(:sales_ymd) |>
    combine(:amount => sum) |>
    orderby(:sales_ymd) |>
    transform(amount_diff = vcat([missing], diff(:amount_sum))) |>
    first(10)

### 042

In [ ]:
# シフトするための関数を定義。
function shift(x::Array, delta::Int)
    return vcat([missing for delta in 1:delta], x[begin:end-delta])
end
function shift(df::DataFrame, delta::Int; rename::Bool=true)
    newdf = DataFrame(shift.(eachcol(df), delta))
    if rename
        newnames = [string("lag_", name, "_", delta) for name in names(df)]
    else
        newnames = [string("lag_", name) for name in names(df)]
    end
    rename!(newdf, newnames)
    return newdf
end

In [ ]:
# 縦横共通の下準備
df = @linq df_receipt |>
    select(:sales_ymd, :amount) |>
    groupby(:sales_ymd) |>
    combine(:amount => sum) |>
    orderby(:sales_ymd);

In [ ]:
# 縦持ちケース
dfs = [hcat(df, shift(df, i, rename=false)) for i in 1:3]
first(dropmissing(vcat(dfs[1], dfs[2], dfs[3])), 10)

In [ ]:
# 横持ちケース
dropmissing(hcat(df, shift(df, 1), shift(df, 2), shift(df, 3)))

### 043

In [ ]:
# 一よりも上の位で切り捨てる関数を用意
# ÷はPythonの//に相当する商を求める演算
function digitfloor(x::Real, digit::Int64)
    d = digit
    return Int(floor(x÷10^d)*10^d)
end

In [ ]:
# あとは適当に処理してunstackするだけ
@linq innerjoin(df_receipt, df_customer, on=:customer_id) |>
    select(:gender_cd, :age, :amount) |>
    transform(age_cd = digitfloor.(:age, 1)) |>
    groupby([:gender_cd, :age_cd]) |>
    combine(:amount => sum) |>
    unstack(:age_cd, :gender_cd, :amount_sum) |>
    rename([:age_cd, :male, :female, :unknown])

### 044

In [ ]:
# verboseだけど愚直に変換するための関数を用意
function convert_cd(x)
    if x == "0"
        return "00"
    elseif x == "1"
        return "01"
    else
        return "99"
    end
end

@linq innerjoin(df_receipt, df_customer, on=:customer_id) |>
    select(:gender_cd, :age, :amount) |>
    transform(age_cd = digitfloor.(:age, 1)) |>
    groupby([:age_cd, :gender_cd]) |>
    combine(:amount => sum) |>
    transform(gender_cd = convert_cd.(:gender_cd)) |>
    orderby(:age_cd, :gender_cd)

### 045

In [ ]:
# string., year., month., day.とすべてにドットをつける必要があることに注意
@linq df_customer |>
    transform(birth_day = string.(Dates.year.(:birth_day),
                                  Dates.month.(:birth_day),
                                  Dates.day.(:birth_day))) |>
    select(:customer_id, :birth_day) |>
    first(10)

### 046

In [ ]:
# パーサーにフォーマットを教えてやるだけでOK
@linq df_customer |>
    select(:customer_id, :application_date) |>
    transform(application_date = Date.(:application_date, "yyyymmdd")) |>
    first(10)

### 047

In [ ]:
# スマートではないがいったんstringに変換
# 速度重視なら÷10000して年、÷100して月（年月）を取り出すとかがベター
@linq df_receipt |>
    select(:receipt_no, :receipt_sub_no, :sales_ymd) |>
    transform(sales_ymd = Date.(string.(:sales_ymd), "yyyymmdd")) |>
    first(10)

### 048

In [ ]:
# そのままの関数があるので使うだけだが、そうするとdatetimeになってしまうので別途dateに変換しておく。
@linq df_receipt |>
    select(:receipt_no, :receipt_sub_no, :sales_epoch) |>
    transform(sales_epoch = Date.(unix2datetime.(:sales_epoch))) |>
    first(10)

### 049

In [ ]:
# ほとんど前問と同じで対応できる
@linq df_receipt |>
    select(:receipt_no, :receipt_sub_no, :sales_epoch) |>
    transform(sales_epoch = Dates.year.(unix2datetime.(:sales_epoch))) |>
    first(10)

### 050

In [ ]:
# 0埋めするためにlpadを使う
@linq df_receipt |>
    select(:receipt_no, :receipt_sub_no, :sales_epoch) |>
    transform(sales_epoch = lpad.(Dates.month.(unix2datetime.(:sales_epoch)), 2, "0")) |>
    first(10)